# Sea Surface Height — NEMO GYRE

Analyse SSH (`sossheig`) from the 6-month GYRE simulation.
- Temporal variance (spatial map)
- Domain-mean SSH time series

In [ ]:
from pathlib import Path

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import nc_time_axis  # noqa: F401 — registers cftime support in matplotlib
import xarray as xr

plt.rcParams.update({"font.size": 16, "axes.titlesize": 18, "axes.labelsize": 14})

OUTPUT_DIR = Path("../output")

## Load simulation output

Load the recombined grid_T file (SSH, SST, SSS) and the mesh_mask
(grid geometry and land/sea mask). Cell widths `e1t`/`e2t` from the
mesh_mask are used for area-weighted spatial averaging.

In [ ]:
# Grid_T output — time axis decoded via CF-compliant 360_day calendar
ds = xr.open_mfdataset(sorted(OUTPUT_DIR.glob("*_grid_T.nc")))

# Mesh mask (time dim is degenerate, skip time decoding)
mask_ds = xr.open_dataset(OUTPUT_DIR / "mesh_mask.nc", decode_times=False)

# Surface ocean mask, excluding the outermost boundary ring
tmask_sfc = mask_ds.tmask.isel(time_counter=0, nav_lev=0)
ny, nx = tmask_sfc.sizes["y"], tmask_sfc.sizes["x"]
not_boundary = (
    (xr.DataArray(range(ny), dims="y") > 0)
    & (xr.DataArray(range(ny), dims="y") < ny - 1)
    & (xr.DataArray(range(nx), dims="x") > 0)
    & (xr.DataArray(range(nx), dims="x") < nx - 1)
)
interior = tmask_sfc.where(not_boundary, 0)

# Cell area (m²) for area-weighted averaging
cell_area = (mask_ds.e1t * mask_ds.e2t).isel(time_counter=0)

# Map projection centred on the GYRE domain
MARGIN = 0.5
proj = ccrs.Stereographic(central_longitude=-68, central_latitude=32)
extent = [
    float(ds.nav_lon.min()) - MARGIN, float(ds.nav_lon.max()) + MARGIN,
    float(ds.nav_lat.min()) - MARGIN, float(ds.nav_lat.max()) + MARGIN,
]

ssh = ds["sossheig"]
ssh

## SSH temporal variance

Variance over time at each grid point — highlights regions with the most
sea-level variability (western boundary current).

In [ ]:
ssh_var = ssh.var("time_counter").where(interior)

fig, ax = plt.subplots(figsize=(8, 7), subplot_kw=dict(projection=proj))
ax.set_extent(extent, crs=ccrs.PlateCarree())
ax.add_feature(cfeature.LAND, facecolor="tan", edgecolor="k", linewidth=0.5)
ax.add_feature(cfeature.COASTLINE, linewidth=0.5)
ax.gridlines(draw_labels=False, alpha=0.3)

pcm = ax.pcolormesh(
    ds.nav_lon.values, ds.nav_lat.values, ssh_var.values,
    shading="auto", cmap="inferno", transform=ccrs.PlateCarree(),
)
fig.colorbar(pcm, ax=ax, label="SSH variance (m²)", shrink=0.7)
ax.set_title("SSH Variance")
fig.tight_layout()
fig.savefig("../figures/ssh_variance.png", dpi=150, bbox_inches="tight")

## Domain-mean SSH time series

Area-weighted SSH averaged over interior ocean points. With `key_linssh`
(linear free surface) the domain volume is conserved exactly, so the
domain-mean SSH stays near zero — any signal is numerical noise at
machine precision.

In [ ]:
weights = cell_area * interior
ssh_mean = ssh.weighted(weights).mean(["y", "x"])

fig, ax = plt.subplots(figsize=(8, 4))
ax.plot(ssh_mean.time_counter, ssh_mean)
ax.set_xlabel("Time")
ax.set_ylabel("Domain-mean SSH (m)")
ax.set_title("NEMO GYRE — Domain-mean Sea Surface Height")
fig.tight_layout()